In [21]:
import os
import pickle
import numpy as np
import pandas as pd
import re


from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [22]:
# --- PARAMETRI PRINCIPALI ---
dist_type = "gaussian"
window_sizes_to_analyze = [450, 300, 150]
strength_threshold = 0.04
tree_max_depth = 5       

output_base_dir = "../result/ltl_rules"
os.makedirs(output_base_dir, exist_ok=True)

In [23]:
try:
    with open("../result/causal_results_gaussian_parcorr/all_causal_results_parcorr.pkl", "rb") as f:
        all_causal_results = pickle.load(f)
    print("File load with success ! ")
except FileNotFoundError:
    print("ERROR: File doesn't exist !")

activities = ['Walk', 'Run', 'Jump', 'Sitting', 'Empty Room']

def get_stable_links(results_dict, activity, window, alpha=0.01, strength_thresh=0.3):
    # take the stable link for a specific activity, making aggregation for the seed result
    seeds = results_dict[window].keys()
    link_counts = {}

    for seed in seeds:
        if activity not in results_dict[window][seed] or 'LPCMCI' not in results_dict[window][seed][activity]:
            continue
            
        lpcmci_results = results_dict[window][seed][activity]['LPCMCI']
        graph = lpcmci_results['graph']
        p_matrix = lpcmci_results['p_matrix']
        val_matrix = lpcmci_results['val_matrix']

        for i in range(graph.shape[0]):
            for j in range(graph.shape[1]):
                for lag in range(graph.shape[2]):
                    # we will take only the strong link and positive for semplicity
                    if p_matrix[i, j, lag] < alpha and val_matrix[i, j, lag] > strength_thresh:
                        link = f'LF_{i}(t{lag-graph.shape[2]}) -> LF_{j}(t)'
                        link_counts[link] = link_counts.get(link, 0) + 1
    
    stable_links = {link for link, count in link_counts.items() if count >= 2}
    return stable_links

File load with success ! 


In [26]:
def parse_decision_tree_rules(tree_rules_text, class_names):
    """
    Parses the text output of export_text to extract the rule conditions for each class.
    """
    rules_per_class = {name: [] for name in class_names}
    path_stack = []

    for line in tree_rules_text.strip().split('\n'):
        indentation = line.find('|---')
        
        # Adjust the stack to the current indentation level
        while len(path_stack) > indentation // 4:
            path_stack.pop()

        # Check if it's a leaf node (a final class decision)
        if 'class:' in line:
            class_name = class_names[int(re.search(r'class: (\d+)', line).group(1))]
            # A rule is defined by the path of conditions to reach this class
            rules_per_class[class_name].append(list(path_stack))
        else:
            # It's a condition, add it to the current path
            # We also determine if the condition is for presence (> 0.5) or absence (<= 0.5)
            condition_text = line.split('|--- ')[1]
            if '<=' in condition_text:
                link_name = condition_text.split(' <= ')[0].strip()
                path_stack.append({'link': link_name, 'present': False})
            else:
                link_name = condition_text.split(' > ')[0].strip()
                path_stack.append({'link': link_name, 'present': True})
                
    return rules_per_class

def translate_rule_to_ltl(rule_conditions):
    """
    Translates a list of rule conditions into a single LTL formula.
    """
    if not rule_conditions:
        return "True" # No specific rule

    ltl_fragments = []
    for condition in rule_conditions:
        link = condition['link']
        
        # Parse the link string e.g., "LF_i(t-1) -> LF_j(t)"
        match = re.match(r'(\w+)\(t-(\d+)\) -> (\w+)\(t\)', link)
        if not match:
            continue
            
        source, lag_str, target = match.groups()
        lag = int(lag_str)
        
        # Create the "next" operators (○) based on the lag
        next_operators = '○' * lag
        
        # Create the core LTL fragment
        ltl_core = f"◻({source} ⇒ {next_operators}{target})"
        
        # Add negation if the rule is for the *absence* of the link
        if not condition['present']:
            ltl_fragments.append(f"¬({ltl_core})")
        else:
            ltl_fragments.append(ltl_core)
            
    # Combine all fragments with AND
    return " ∧ ".join(ltl_fragments)



In [28]:
for window in window_sizes_to_analyze:
    print(f"--- Start Analysis by Window Size: {window} ---")
    
    window_output_dir = os.path.join(output_base_dir, f"window_{window}")
    os.makedirs(window_output_dir, exist_ok=True)
    
    stable_links_per_activity = {
        activity: get_stable_links(
            all_causal_results,
            activity=activity,
            window=window,
            strength_thresh=strength_threshold
        ) for activity in activities
    }
    
    master_link_list = sorted(list(set.union(*stable_links_per_activity.values())))
    
    if not master_link_list:
        print(f"No stable link found for window {window}. I'll move on to the next one.\n")
        continue
        
    print(f" Found {len(master_link_list)} Stable unique window ties {window}.")
    
    X = np.array([[1 if link in stable_links_per_activity[act] else 0 for link in master_link_list] for act in activities])
    y_labels = np.array(activities)
    
    causal_feature_df = pd.DataFrame(X, columns=master_link_list, index=y_labels)

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_labels)
    
    clf = DecisionTreeClassifier(max_depth=tree_max_depth, random_state=42)
    clf.fit(X, y_encoded)
    
    feature_names = causal_feature_df.columns.tolist()
    tree_rules_text = export_text(clf, feature_names=feature_names)
    
    tree_structure_path = os.path.join(window_output_dir, "_tree_structure.txt")
    with open(tree_structure_path, "w", encoding="utf-8") as f:
        f.write(tree_rules_text)
    print(f"Tree structure saved in: {os.path.join(window_output_dir, '_tree_structure.txt')}")
        
    parsed_rules = parse_decision_tree_rules(tree_rules_text, y_encoded)
    
    print("Salvataggio delle regole LTL per ogni attività...")
    for class_int, conditions_list in parsed_rules.items():
        class_name = label_encoder.inverse_transform([class_int])[0]
        if conditions_list:
            
        
            activity_output_dir = os.path.join(window_output_dir, class_name.replace(' ', '_'))
            os.makedirs(activity_output_dir, exist_ok=True)

            ltl_formula = translate_rule_to_ltl(conditions_list[0])
            rule_text = f"Activity: {class_name}\nFormula LTL: {class_name} ⇔ {ltl_formula}"
            
            rule_filename = "ltl_rule.txt"
            rule_filepath = os.path.join(activity_output_dir, rule_filename)
            with open(rule_filepath, "w",encoding="utf-8") as f:
                f.write(rule_text)
            print(f" > Regola per '{class_name}' saved in: {rule_filepath}")
            
    print(f"--- Analyze for Window Size: {window} completed ---\n")



--- Start Analysis by Window Size: 450 ---
 Found 35 Stable unique window ties 450.
Tree structure saved in: ../result/ltl_rules\window_450\_tree_structure.txt
Salvataggio delle regole LTL per ogni attività...
 > Regola per 'Walk' saved in: ../result/ltl_rules\window_450\Walk\ltl_rule.txt
 > Regola per 'Run' saved in: ../result/ltl_rules\window_450\Run\ltl_rule.txt
 > Regola per 'Jump' saved in: ../result/ltl_rules\window_450\Jump\ltl_rule.txt
 > Regola per 'Sitting' saved in: ../result/ltl_rules\window_450\Sitting\ltl_rule.txt
 > Regola per 'Empty Room' saved in: ../result/ltl_rules\window_450\Empty_Room\ltl_rule.txt
--- Analyze for Window Size: 450 completed ---

--- Start Analysis by Window Size: 300 ---
 Found 69 Stable unique window ties 300.
Tree structure saved in: ../result/ltl_rules\window_300\_tree_structure.txt
Salvataggio delle regole LTL per ogni attività...
 > Regola per 'Walk' saved in: ../result/ltl_rules\window_300\Walk\ltl_rule.txt
 > Regola per 'Run' saved in: ../res